在本教程中，我們將學習如何使用 **梯度提升(gradient boosting)** 來構建和優化模型。這種方法在許多Kaggle競賽中占主導地位，並在各種數據集上獲得了最新穎的結果。

# 介紹

在本課程的大部分時間裡，您都是使用隨機森林方法進行預測的，該方法比平均單個決策樹的預測結果要好，因此它比單個決策樹的性能更好。

我們將隨機森林方法稱為“集成方法(ensemble method)”。根據定義， **集成方法** 結合了幾種模型的預測（例如，對於隨機森林而言，是幾棵樹）。

接下來，我們將學習另一種稱為梯度增強的集成方法。

# 梯度提升

**梯度提升** 是一種經過循環將模型迭代添加到集合中的方法。

首先從用單個模型初始化開始，其預測可能很差。 （即使其預測非常不准確，隨後對該集合進行的調整也將修正這些錯誤。）

然後，我們開始循環：

- 首先，我們使用當前集合為數據集中的每個觀測值生成預測。為了做出預測，我們將集合中所有模型的預測相加。

- 這些預測用於計算損失函數（例如[均方誤差]（https://en.wikipedia.org/wiki/Mean_squared_error））。

- 然後，我們使用損失函數來擬合將添加到集合中的新模型。具體來說，我們確定模型參數，以便將此新模型添加到集合中將減少損失。 （*備註：“梯度提升”中的“梯度”是指我們將對損失函數使用[梯度下降]（https://en.wikipedia.org/wiki/Gradient_descent）來確定新模型中的參數。*）

- 最後，我們將新模型添加到集合中，然後...
- ... 重複！

# 範例

我們首先將訓練和驗證數據加載到 `X_train` ， `X_valid` ， `y_train` 和 `y_valid` 中。

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Read the data
data = pd.read_csv('./data_set/melb_data.csv')

# Select subset of predictors
cols_to_use = ['Rooms', 'Distance', 'Landsize', 'BuildingArea', 'YearBuilt']
X = data[cols_to_use]

# Select target
y = data.Price

# Separate data into training and validation sets
X_train, X_valid, y_train, y_valid = train_test_split(X, y)

在此示例中，我們將使用 XGBoost函式庫。 **XGBoost** 代表 **極端梯度增強(extreme gradient boosting)**，它是梯度增強的一種實現，具有一些額外的功能，這些性能側重於性能和速度。 （ *Scikit-learn 有梯度增強的另一個版本，但是XGBoost具有一些技術優勢。* ）

在下一個代碼單元中，我們導入用於XGBoost的scikit-learn API（`xgboost.XGBRegressor`)。 這使我們能夠像在scikit-learn中一樣構建和擬合模型。 正如您將在輸出中看到的那樣，`XGBRegressor` 函式具有許多可調參數，我們將很快了解它們！

In [2]:
from xgboost import XGBRegressor

my_model = XGBRegressor()
my_model.fit(X_train, y_train)

C:\Users\user\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[15:51:39] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

我們也進行預測並評估模型。

In [4]:
from sklearn.metrics import mean_absolute_error

predictions = my_model.predict(X_valid)
print("Mean Absolute Error: " + str(mean_absolute_error(predictions, y_valid)))

Mean Absolute Error: 267419.6959131075


# 參數調整

XGBoost具有一些可以極大地影響準確性和訓練速度的參數。 您應該了解的第一個參數是：

### n_estimators
`n_estimators` 指定要經過上述建模循環的次數。 它等於我們包含在集合中的模型數量。

-值太低會導致*擬合不足(underfitting)*，導致訓練數據和測試數據的預測不准確。
-值太高會導致*overfitting*，這會導致對訓練數據的準確預測，但是對測試數據的預測不准確（ *這是我們關心的* ）。

典型值範圍是100-1000，儘管這在很大程度上取決於下面討論的 `learning_rate` 參數。

這是設置整體模型數量的程式碼：

In [5]:
my_model = XGBRegressor(n_estimators=500)
my_model.fit(X_train, y_train)

C:\Users\user\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[16:19:21] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=500,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

### early_stopping_rounds

 `early_stopping_rounds` 提供了一種自動找到 `n_estimator` 理想值的方法。 儘早停止會使模型在驗證分數停止提高時停止迭代，即使我們不是硬停n_estimators的。 為n_estimators設置一個較高的值，然後使用early_stopping_rounds查找停止迭代的最佳時間是很明智的。

由於隨機機會(random chance)有時會導致驗證分數沒有提高的單回合，因此您需要指定一個數字，以便在停止前允許連續回合惡化的次數。 設置 `early_stopping_rounds = 5` 是一個合理的選擇。 在這種情況下，我們會在連續5輪驗證評分下降後停止。

使用 `early_stopping_rounds` 時，您還需要預留一些數據來計算驗證分數-這是通過設置 `eval_set` 參數來完成的。

我們可以修改上面的示例以包括提前停止：

In [6]:
my_model = XGBRegressor(n_estimators=500)
my_model.fit(X_train, y_train, 
             early_stopping_rounds=5, 
             eval_set=[(X_valid, y_valid)],
             verbose=False)

[16:23:35] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=500,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

如果之後要用所有數據擬合模型，請將n_estimators設置為在進行早期停止運行時發現的最佳值。

### 學習率

與其簡單地將每個組件模型中的預測相加即可得到預測，我們還可以在將每個模型中的預測相加之前將其乘以一個小數（稱為學習率）。

這意味著我們添加到集合中的每棵樹對我們的幫助都會有所減少。 因此，我們可以為n_estimators設置更高的值而不會過度擬合。 如果我們使用提前停止，則將自動確定適當的樹木數量。

通常，小的學習率和大量的估計量將產生更準確的XGBoost模型，儘管由於在整個循環中進行了更多的迭代，所以模型的訓練時間也更長。 默認情況下，XGBoost設置 `learning_rate = 0.1` 。

修改上面的示例以更改學習率，將產生以下程式碼：

In [7]:
my_model = XGBRegressor(n_estimators=1000, learning_rate=0.05)
my_model.fit(X_train, y_train, 
             early_stopping_rounds=5, 
             eval_set=[(X_valid, y_valid)], 
             verbose=False)

[16:27:26] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.05, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=1000,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

### n_jobs

在考慮運行時的較大數據集上，可以使用並行性更快地構建模型。 通常將參數 `n_jobs` 設置為等於電腦上的內核數。 在較小的數據集上，這無濟於事。

最終的模型結果不會更好，因此對擬合時間進行微優化通常只是分散注意力。 但是，它在大型數據集中很有用，否則您將需要花費很長時間在fit命令中等待。

這是修改後的示例：

In [8]:
my_model = XGBRegressor(n_estimators=1000, learning_rate=0.05, n_jobs=4)
my_model.fit(X_train, y_train, 
             early_stopping_rounds=5, 
             eval_set=[(X_valid, y_valid)], 
             verbose=False)

[16:31:35] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.05, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=1000,
             n_jobs=4, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

# 結論

[XGBoost](https://xgboost.readthedocs.io/en/latest/)是用於處理標準表格數據（存儲在Pandas DataFrames中的數據類型，而不是圖像和視頻等更奇特的數據類型）。 通過仔細的參數調整，您可以訓練高度準確的模型。